In [16]:
import load_data as ld

Loader = ld.DataLoader()

train_df, val_df, test_df = Loader.load_all_three_dfs(max_size=126171)

In [17]:
embedder = pe.PretrainedEmbedder("esm1_t6_43M_UR50S")

In [24]:
esm_train = embedder.embed_sequences(input_df=train_df)

  0%|          | 250/126171 [01:05<10:16:43,  3.40it/s]

In [ ]:
esm_val= embedder.embed_sequences(input_df=val_df)
esm_test = embedder.embed_sequences(input_df=test_df)